In [1]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os
import json
from importlib import reload
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')

# 3rd party lib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits

# my lib
import ImageTools as it
import DataTools as dt
import gphelper
from gphelper import GPHelper

Using matplotlib backend: Qt5Agg


In [7]:
gp_h = GPHelper(restore_file='gp.json')
gp_j = GPHelper(restore_file='gp_j.json')
gp_v = GPHelper(restore_file='gp_v.json')
gp_z = GPHelper(restore_file='gp_z.json')

rp_ip = []
with open('rp_ip', 'r') as f:
    for line in f:
        rp_ip.append(line.strip())


In [37]:
# draw a sample
rpip = np.random.choice(rp_ip)
rpip = rpip.split(',')
rp, ip = float(rpip[0]), float(rpip[1])

y, x = np.meshgrid(np.arange(84), np.arange(84))
cx, cy = 41.5, 41.25
rs = np.sqrt((x-cx)**2 + (y-cy)**2) 
norm_rs = rs / rp
img = norm_rs.copy()

sorted_rs = np.array(sorted(np.unique(img)))
fs = 10**gp_h.sample(sorted_rs[sorted_rs<=5,np.newaxis]) * ip

for r in sorted_rs:
    if r <= 5:
        img[img==r] = fs[sorted_rs[sorted_rs<=5]==r, 0]
    else:
        img[img==r] = 0

plt.imshow(img)

In [49]:
# get color ratio
color_rs = np.array(sorted(np.unique(rs[norm_rs<=5])))

pixel_bound = 15
j_vals = gp_j.predict(color_rs[color_rs<=pixel_bound,np.newaxis])
v_vals = gp_v.predict(color_rs[color_rs<=pixel_bound,np.newaxis])
z_vals = gp_z.predict(color_rs[color_rs<=pixel_bound,np.newaxis])

j_ratio = rs.copy()
v_ratio = rs.copy()
z_ratio = rs.copy()

for r in color_rs:
    if r <= pixel_bound:
        j_ratio[rs==r] = j_vals[color_rs[color_rs<=pixel_bound]==r]
        v_ratio[rs==r] = v_vals[color_rs[color_rs<=pixel_bound]==r]
        z_ratio[rs==r] = z_vals[color_rs[color_rs<=pixel_bound]==r]
    else:
        j_ratio[rs==r] = 1.0
        v_ratio[rs==r] = 1.0
        z_ratio[rs==r] = 1.0

j_img = j_ratio * img     
v_img = v_ratio * img
z_img = z_ratio * img

plt.figure()
plt.imshow(j_img)
plt.figure()
plt.imshow(v_img)
plt.figure()
plt.imshow(z_img)

In [53]:
# get noise
noise_params = []
with open('signal_to_noise', 'r') as f:
    for line in f:
        #noise_params.append(tuple(float(i) for i in line.strip().split(',')))
        noise_params.append(line.strip())

h_mean, h_std, h_snr, j_mean, j_std, j_snr, v_mean, v_std, v_snr, z_mean, z_std, z_snr = tuple(float(i) for i in np.random.choice(noise_params).split(','))
h_noise = np.random.normal(loc=h_mean, scale=h_std, size=[84,84])
j_noise = np.random.normal(loc=j_mean, scale=j_std, size=[84,84])
v_noise = np.random.normal(loc=v_mean, scale=v_std, size=[84,84])
z_noise = np.random.normal(loc=z_mean, scale=z_std, size=[84,84])

plt.figure()
plt.imshow(h_noise)
plt.figure()
plt.imshow(j_noise)
plt.figure()
plt.imshow(v_noise)
plt.figure()
plt.imshow(z_noise)

In [54]:
# adjust snr to a measured level
scaled_h = h_noise * (h_snr/(img.mean()/h_noise.mean()))
scaled_j = j_noise * (j_snr/(img.mean()/j_noise.mean()))
scaled_v = v_noise * (v_snr/(img.mean()/v_noise.mean()))
scaled_z = z_noise * (z_snr/(img.mean()/z_noise.mean()))

plt.figure()
plt.title('H')
plt.imshow(img+h_noise)
plt.figure()
plt.title('J')
plt.imshow(j_img+j_noise)
plt.figure()
plt.title('V')
plt.imshow(v_img+v_noise)
plt.figure()
plt.title('Z')
plt.imshow(z_img+z_noise)
